In [1]:
import utils
import importlib

importlib.reload(utils)

[2023-09-13 17:07:19,774] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


<module 'utils' from '/data/users/zhangjunlei/tyx/reward-by-prm800k/src/utils.py'>

In [2]:
from transformers import LlamaForCausalLM, GenerationConfig
import random
import torch
import os
from collections import defaultdict
from tqdm import tqdm

In [3]:
utils.init_logging()
logger = utils.get_logger(__name__)
logger.info("test")

2023-09-13 17:07:20.802 [INFO] test
2023-09-13 17:07:20.802 [INFO] test


<Logger __main__ (INFO)>


In [4]:
# utils.set_gpu_ids([6])
gpu_id = "7"
device = f"cuda:{gpu_id}"

In [5]:
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path)
# tokenizer = LlamaTokenizer.from_pretrained(tokenizer_name_or_path)
tokenizer = utils.get_complete_tokenizer()

head_tokens = <unk><s></s> 
tokenizer.vocab_size = 32000
len(tokenizer.vocab) = 32000
tokenizer.special_tokens_map = {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}
num_added_tokens = 1
head_tokens = <unk><s></s> 
tokenizer.vocab_size = 32000
len(tokenizer.vocab) = 32001
tokenizer.special_tokens_map = {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}
LlamaTokenizerFast(name_or_path='/data/users/zhangjunlei/tyx/.cache/huggingface/hub/models--hf-internal-testing--llama-tokenizer/snapshots/99eceeba6e8289bee767f0771166b5917e70e470', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}


In [6]:
model = LlamaForCausalLM.from_pretrained(
    utils.default_7b_model_path, low_cpu_mem_usage=True
).to(device)

embedding_size = model.get_input_embeddings().weight.shape[0]
if len(tokenizer) > embedding_size:
    model.resize_token_embeddings(len(tokenizer))

print(model)
print(model.config)
print(model.generation_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32001, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_

In [7]:
def sparse_print(*arg, **kwargs):
    print(*arg, **kwargs, sep="\n", end="\n\n")


def generate(prompt_or_list, generation_config):
    if not isinstance(generation_config, GenerationConfig):
        if isinstance(generation_config, dict):
            generation_config = GenerationConfig.from_dict(generation_config)

    inputs = tokenizer(prompt_or_list, padding=True, return_tensors="pt").to(device)
    sparse_print("inputs:")
    sparse_print(inputs)

    outputs = model.generate(**inputs, generation_config=generation_config)
    sparse_print(outputs)

    if not generation_config.return_dict_in_generate:
        decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        sparse_print(decoded_outputs)

        return outputs
    else:  # generation_config.return_dict_in_generate == True
        outputs_len = outputs["sequences"].shape[0]

        example_idx = random.randint(0, outputs_len - 1)
        sparse_print(f"example_idx = {example_idx}")

        if outputs_len > 5:
            example_decoded_output = tokenizer.decode(
                outputs["sequences"][example_idx], skip_special_tokens=True
            )
            sparse_print(example_decoded_output)
        else:  # outputs_len <= 5
            decoded_outputs = tokenizer.batch_decode(
                outputs["sequences"], skip_special_tokens=True
            )
            sparse_print(decoded_outputs)

        if generation_config.output_scores:
            example_scores = outputs["scores"][example_idx]
            sparse_print(example_scores.shape)
            sparse_print(example_scores)

            example_probs = torch.nn.functional.softmax(example_scores, dim=-1)
            sparse_print(example_probs[example_idx])
            sparse_print(sum(example_probs[example_idx]))

            # slow cpu
            # for idx, prob in enumerate(first_probs[0]):
            #     if prob > 1e-5:
            #         print(f"token_id: {idx}, token: {tokenizer.decode([idx])}, prob: {prob}")

            # fast gpu
            # non_zero_idxs = torch.gt(first_probs.view(-1),1e-1)
            threshold = 0.1
            mask = example_probs > threshold
            non_zero_idxs = torch.nonzero(mask)
            sparse_print(non_zero_idxs)
            non_zero_vals = example_probs.view(-1)[non_zero_idxs]
            sparse_print(non_zero_vals)

        return outputs

In [8]:
# test
generation_config = {
    "do_sample": True,
    # "temperature": 0.8,
    # "top_p": 0.95,
    "temperature": 0.7,
    "top_p": 0.9,
    "max_new_tokens": 10,
    "return_dict_in_generate": True,
}

prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

outputs = generate(prompt_or_list=prompts, generation_config=generation_config)

decoded_outputs = tokenizer.batch_decode(**outputs, skip_special_tokens=True)

for prompt, decoded_output in zip(prompts, decoded_outputs):
    print(f"Prompt    : {prompt}")
    print(f"Generation: {decoded_output[len(prompt):]}")
    print()

inputs:

{'input_ids': tensor([[32000, 32000,     1, 15043, 29892,   590,  1024,   338],
        [    1,   450,  6673,   310,   278,  3303,  3900,   338],
        [32000, 32000,     1,   450,  7483,   310,  3444,   338],
        [32000,     1,   450,  5434,   310,   319, 29902,   338]],
       device='cuda:7'), 'attention_mask': tensor([[0, 0, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 1, 1, 1]], device='cuda:7')}

SampleDecoderOnlyOutput(sequences=tensor([[32000, 32000,     1, 15043, 29892,   590,  1024,   338,   476,  2486,
         29876, 29889,   306,   626,   263, 20183,   472,  8068],
        [    1,   450,  6673,   310,   278,  3303,  3900,   338,   278,  1556,
         13988,  2022,   297,   278,  3186, 29889,  2193, 29915],
        [32000, 32000,     1,   450,  7483,   310,  3444,   338,   697,   310,
           278,  1556,  9560, 14368,   297,   278,  3186, 29889],
        [32000,     1,   450,  5434,   310, 

In [9]:
# debug
# print(generation_config.return_dict_in_generate)

In [10]:
problem = "A $90^\\circ$ rotation around the origin in the counter-clockwise direction is applied to $7 + 2i.$  What is the resulting complex number?"
steps = [
    "I know that a $90^\\circ$ rotation around the origin in the complex plane can be achieved by multiplying the complex number by $i$, since $i$ has a magnitude of 1 and an argument of $90^\\circ$.",
    "So, I can write the rotation as $(7 + 2i)i = 7i + 2i^2.$",
    "To simplify this expression, I recall that $i^2 = -1$, so I can substitute that and get $7i - 2.$",
    "This is the resulting complex number after the rotation.\n\n# Answer\n\n7i - 2",
]

rating2word = {1: "positive", -1: "negative", 0: "neutral"}
rating_words = list(rating2word.values())
rating_token_ids = tokenizer(rating_words, add_special_tokens=False).input_ids
print(rating_token_ids)

# constraint = transformers.Constraint(type="must_include", token="apple")

# config
generation_config = {
    "do_sample": False,
    "max_new_tokens": 1,
    "force_words_ids": rating_token_ids,
    "num_beams": 2,
    "remove_invalid_values": True,
    "return_dict_in_generate": True,
    "output_scores": True,
}

[[6374], [8178], [21104]]


In [11]:
problem_solution_hierchical_samples = utils.load_pickle("/data/users/zhangjunlei/tyx/reward-by-prm800k/datasets/problem-solution-hierarchical-samples.pkl")

In [27]:
debug = {
    # "sample": True,
    # "prompt": True,
    # "problem": True,
    # "step": True
}

In [16]:
problem2steps_prompts = defaultdict(list)

for problem_solution_samples in problem_solution_hierchical_samples:
    problem = problem_solution_samples["problem"]
    for solution_sample in problem_solution_samples["solutions"]:
        if debug.get("sample"):
            print(solution_sample)
            raise Exception
        # reformatted_sample = utils.reformat_prm800k_sample(solution_sample) # different format
        # prompt = problem
        step_prompt = "\n".join(solution_sample["steps"])
        problem2steps_prompts[problem].append(step_prompt)

In [28]:
problem_step_ids_list = []
for problem, step_prompts in tqdm(problem2steps_prompts.items()):
    problem_step_ids = {
        "problem_ids": None,
        "step_prompt_ids": []
    }
    problem_ids = tokenizer(problem + "\n", add_special_tokens=True).input_ids
    problem_step_ids["problem_ids"] = problem_ids
    if debug.get("problem"):
        print(problem)
        print(problem_ids)
        raise Exception
    for step_prompt in step_prompts:
        step_prompt_ids = tokenizer(step_prompt, add_special_tokens=False).input_ids
        problem_step_ids["step_prompt_ids"].append(step_prompt_ids)
        if debug.get("step"):
            print(step_prompt)
            print(step_prompt_ids)
            raise Exception
        
print(len(problem_step_ids_list))

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2457624929.py, line 4)

In [14]:
# validation set
# validation_set_path = os.path.join(utils.project_root, "datasets", "prm800k-002validation-seed42.jsonl")
# validation_set = utils.load_jsonl(validation_set_path)
# print(len(validation_set))
# print(validation_set[0])

In [15]:
generate(problem + steps[0], generation_config=generation_config)

inputs:

{'input_ids': tensor([[    1,  2803,   395, 29888, 29898, 29916,  3892, 29989, 29916, 29899,
         29886, 29989, 29974, 29989, 29916, 29899, 29896, 29945, 29989, 29974,
         29989, 29916, 29899, 29886, 29899, 29896, 29945, 29989,  8209,   988,
           395, 29900,   529,   282,   529, 29871, 29896, 29945,  7449,  5953,
           837,   457,   278,  9212,   995,  4586,   491,   395, 29888, 29898,
         29916,  1262,   363,   395, 29916, 29938,   297,   278,  7292,   395,
         29886,   320,  3797,   921, 29905,  3797, 29896, 29945,  7449, 29902,
          1073,   393,   263,   395, 29929, 29900,  3823,  6034, 29938, 13733,
          2820,   278,  3978,   297,   278,  4280, 10694,   508,   367, 14363,
           491,  6674,  5890,   278,  4280,  1353,   491,   395, 29875,  1628,
          1951,   395, 29875, 29938,   756,   263, 18497,   310, 29871, 29896,
           322,   385,  2980,   310,   395, 29929, 29900,  3823,  6034,  1504]],
       device='cuda:7'), 'a

BeamSearchDecoderOnlyOutput(sequences=tensor([[    1,  2803,   395, 29888, 29898, 29916,  3892, 29989, 29916, 29899,
         29886, 29989, 29974, 29989, 29916, 29899, 29896, 29945, 29989, 29974,
         29989, 29916, 29899, 29886, 29899, 29896, 29945, 29989,  8209,   988,
           395, 29900,   529,   282,   529, 29871, 29896, 29945,  7449,  5953,
           837,   457,   278,  9212,   995,  4586,   491,   395, 29888, 29898,
         29916,  1262,   363,   395, 29916, 29938,   297,   278,  7292,   395,
         29886,   320,  3797,   921, 29905,  3797, 29896, 29945,  7449, 29902,
          1073,   393,   263,   395, 29929, 29900,  3823,  6034, 29938, 13733,
          2820,   278,  3978,   297,   278,  4280, 10694,   508,   367, 14363,
           491,  6674,  5890,   278,  4280,  1353,   491,   395, 29875,  1628,
          1951,   395, 29875, 29938,   756,   263, 18497,   310, 29871, 29896,
           322,   385,  2980,   310,   395, 29929, 29900,  3823,  6034,  1504,
           306